[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Freinforcement_learning_gym%2Fpendulum_sac%2Fpendulum_sac.ipynb)

You can easily run this notebook at https://lab.mlpack.org/

Here, we train a [Soft Actor-Critic](https://arxiv.org/abs/1801.01290) agent to get high scores for the [Pendulum](https://gym.openai.com/envs/Pendulum-v0/) environment. 

We make the agent train and test on OpenAI Gym toolkit's GUI interface provided through a distributed infrastructure (TCP API). More details can be found [here](https://github.com/zoq/gym_tcp_api).

A video of the trained agent can be seen in the end.

## Including necessary libraries and namespaces

In [1]:
#include <mlpack/xeus-cling.hpp>

In [2]:
#include <mlpack/core.hpp>
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/reinforcement_learning/sac.hpp>
#include <mlpack/methods/ann/loss_functions/empty_loss.hpp>
#include <mlpack/methods/ann/init_rules/gaussian_init.hpp>
#include <mlpack/methods/reinforcement_learning/environment/env_type.hpp>
#include <mlpack/methods/reinforcement_learning/training_config.hpp>

In [3]:
// Used to run the agent on gym's environment (provided externally) for testing.
#include <gym/environment.hpp>

In [4]:
// Used to generate and display a video of the trained agent.
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [5]:
using namespace mlpack;

In [6]:
using namespace mlpack::ann;

In [7]:
using namespace ens;

In [8]:
using namespace mlpack::rl;

## Initializing the agent

In [9]:
// Set up the state and action space.
ContinuousActionEnv::State::dimension = 3;
ContinuousActionEnv::Action::size = 1;

In [10]:
// Set up the actor and critic networks.
FFN<EmptyLoss<>, GaussianInitialization>
    policyNetwork(EmptyLoss<>(), GaussianInitialization(0, 0.1));
policyNetwork.Add(new Linear<>(ContinuousActionEnv::State::dimension, 32));
policyNetwork.Add(new ReLULayer<>());
policyNetwork.Add(new Linear<>(32, ContinuousActionEnv::Action::size));
policyNetwork.Add(new TanHLayer<>());

FFN<EmptyLoss<>, GaussianInitialization>
    qNetwork(EmptyLoss<>(), GaussianInitialization(0, 0.1));
qNetwork.Add(new Linear<>(ContinuousActionEnv::State::dimension +
                          ContinuousActionEnv::Action::size, 32));
qNetwork.Add(new ReLULayer<>());
qNetwork.Add(new Linear<>(32, 1));

In [11]:
// Set up the policy method.
RandomReplay<ContinuousActionEnv> replayMethod(32, 10000);

In [12]:
// Set up training configurations.
TrainingConfig config;
config.TargetNetworkSyncInterval() = 1;
config.UpdateInterval() = 1;

In [13]:
// Set up Soft actor-critic agent.
SAC<ContinuousActionEnv, decltype(qNetwork), decltype(policyNetwork), AdamUpdate>
    agent(config, qNetwork, policyNetwork, replayMethod);

## Preparation for training the agent

In [14]:
// Set up the gym training environment.
gym::Environment env("gym.kurg.org", "4040", "Pendulum-v0");

// Initializing training variables.
std::vector<double> returnList;
size_t episodes = 0;
bool converged = true;

// The number of episode returns to keep track of.
size_t consecutiveEpisodes = 25;

In [15]:
// Function to train the agent on the Pendulum gym environment.
void Train(const size_t numSteps)
{
  agent.Deterministic() = false;
  std::cout << "Training for " << numSteps << " steps." << std::endl;
  while (agent.TotalSteps() < numSteps)
  {
    double episodeReturn = 0;
    env.reset();
    do
    {
      agent.State().Data() = env.observation;
      agent.SelectAction();
      arma::mat action = {double(agent.Action().action[0] * 2)};

      env.step(action);
      ContinuousActionEnv::State nextState;
      nextState.Data() = env.observation;

      replayMethod.Store(agent.State(), agent.Action(), env.reward, nextState,
          env.done, 0.99);
      episodeReturn += env.reward;
      agent.TotalSteps()++;
      if (agent.Deterministic() || agent.TotalSteps() < config.ExplorationSteps())
        continue;
      for (size_t i = 0; i < config.UpdateInterval(); i++)
        agent.Update();
    } while (!env.done);
    returnList.push_back(episodeReturn);
    episodes += 1;

    if (returnList.size() > consecutiveEpisodes)
      returnList.erase(returnList.begin());
        
    double averageReturn = std::accumulate(returnList.begin(),
                                           returnList.end(), 0.0) /
                           returnList.size();
    if(episodes % 4 == 0)
    {
      std::cout << "Avg return in last " << returnList.size()
          << " episodes: " << averageReturn
          << "\t Episode return: " << episodeReturn
          << "\t Total steps: " << agent.TotalSteps() << std::endl;
    }
  }
}

## Let the training begin

In [16]:
// Training the agent for a total of at least 5000 steps.
Train(5000)

Training for 5000 steps.
Avg return in last 4 episodes: -1110.78	 Episode return: -660.74	 Total steps: 800
Avg return in last 8 episodes: -952.778	 Episode return: -838.488	 Total steps: 1600
Avg return in last 12 episodes: -871.959	 Episode return: -688.612	 Total steps: 2400
Avg return in last 16 episodes: -790.431	 Episode return: -129.354	 Total steps: 3200
Avg return in last 20 episodes: -755.523	 Episode return: -1.9158	 Total steps: 4000
Avg return in last 24 episodes: -693.449	 Episode return: -122.304	 Total steps: 4800


## Testing the trained agent

In [17]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "Pendulum-v0");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action[0] * 2)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video

 Total steps: 200	 Total reward: -1183.11
https://gym.kurg.org/3b171aaeb3d74/output.webm

A Jupyter widget

## A little more training...

In [18]:
// Training the same agent for a total of at least 40000 steps.
Train(40000)

Training for 40000 steps.
Avg return in last 25 episodes: -623.784	 Episode return: -134.917	 Total steps: 5600
Avg return in last 25 episodes: -600.928	 Episode return: -243.994	 Total steps: 6400
Avg return in last 25 episodes: -510.564	 Episode return: -234.78	 Total steps: 7200
Avg return in last 25 episodes: -415.94	 Episode return: -0.388022	 Total steps: 8000
Avg return in last 25 episodes: -336.953	 Episode return: -129.554	 Total steps: 8800
Avg return in last 25 episodes: -301.628	 Episode return: -126.201	 Total steps: 9600
Avg return in last 25 episodes: -231.332	 Episode return: -131.894	 Total steps: 10400
Avg return in last 25 episodes: -178.349	 Episode return: -253.626	 Total steps: 11200
Avg return in last 25 episodes: -168.248	 Episode return: -242.201	 Total steps: 12000
Avg return in last 25 episodes: -177.98	 Episode return: -119.084	 Total steps: 12800
Avg return in last 25 episodes: -177.816	 Episode return: -123.397	 Total steps: 13600
Avg return in last 25 epi

# Final agent testing!

In [19]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "Pendulum-v0");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action[0] * 2)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video

 Total steps: 200	 Total reward: -234.036
https://gym.kurg.org/6a964dee60c14/output.webm

A Jupyter widget